In [90]:
import numpy as np
import random
# import matplotlib.pyplot as plt

# 2009年到2018年的上海房均价
x = np.array([0,1,2,3,4,5,6,7,8,9], dtype = np.float32) # 归一化（-2009)
y = np.array([1.8, 2.1, 2.3, 2.3, 2.85, 3.0, 3.3, 4.9, 5.45, 5.0], dtype = np.float32) # 归一化（除10000）

# 继续归一化到（0，1）的范围
x = x / 10
y = y / 10

In [91]:
# 开始给定估计值
k = random.random() # random产出的随机数符合正态分布，恰好符合误差项的分布 # 斜率
b = 0 # 截距 我们从0开始估计
lr = 1e-1 # 学习率策略步长

### 梯度下降法 (Gradient descent)

In [92]:
for i in range(1000):
    predict = k * x + b
    loss = 0.5 * np.sum((y - predict) ** 2) # 最小二乘法
    if not i % 100:
        print("loss:", loss)
    '''
    求loss的极值，对k和b求导
    熟练地话可以直接求出来(看下面的公式)，或者像下面一样拆解
    loss = 0.5 * (y - predict) ** 2
    loss = 0.5 * A
    A = B ** 2
    B = y - predict
    predict = k * x + b

    dp/dk = x
    dp/db = 1
    dB/dp = -1
    dA/dB = 2B
    dl/dA = 0.5

    然后运用Chain-Rule
    => dl/dk = 0.5 * 2B * -1 * x = x * (predict -y)
    => dl/db = 0.5 * 2B * -1 * 1 = predict - y
    '''
    deltak = np.mean((predict - y) * x) # x,y是数组，求平均值
    deltab = np.mean(predict -y)

    # 分别用dk, db，以及合适的步长lr来修正下一次估计
    k = k - lr * deltak
    b = b - lr * deltab

k, b

loss: 0.32300156354904175
loss: 0.01098606362938881
loss: 0.009797893464565277
loss: 0.009492980316281319
loss: 0.009414737112820148
loss: 0.009394658729434013
loss: 0.009389504790306091
loss: 0.009388182312250137
loss: 0.009387842379510403
loss: 0.009387752041220665


(0.41440958234930414, 0.14352012782046622)

$$
J(\theta) = \frac{1}{2} \sum_{i=1}^n(y_i - \theta_0 - \theta_1x_i)^2 \tag{最小二乘法}
$$
分别对$\theta_0$和$\theta_1$求导, 其实就是对$(y_i - \theta_0 - \theta_1x_i)^2$整体求导$(A^2)^\prime= 2A$
再应用链式法则求$(y_i - \theta_0 - \theta_1x_i)$分别对$\theta_0和\theta_1x_i$的导数
> 分别为-1和-x

$$
\frac{\delta J(\theta)}{\delta\theta_0} = 2  \times\frac{1}{2} \sum_{i=1}^n(y_i - \theta_0 - \theta_1x_i) \times (-1)
$$
$$
\frac{\delta J(\theta)}{\delta\theta_1} = 2  \times\frac{1}{2} \sum_{i=1}^n(y_i - \theta_0 - \theta_1x_i) \times (-x)
$$

In [93]:
next_year = 2020
next_price = (k * (next_year - 2009) / 10 + b) * 10
print(f'{next_year}年房价估计：{next_price:.2f}万')

2020年房价估计：5.99万


In [94]:
# 用matplotlib去把点和线绘制一下
# 我这里导入matplotlib.pyplot库就开始出错了（只导入matplotlib不出错）
x1, y1 = 0, k*0+b
x2, y2 = 1, k*1+b
plt.plot(x, y, 'b*')                # 用蓝色绘制点
plt.plot([x1, x2], [y1, y2], 'g-')  # 用绿色绘制线

NameError: name 'plt' is not defined

### 梯度下降法求sqrt(v)

$loss = (x^2 - v)^2$

>这个方法数字到了17就溢出了（计算$x^2-v$时出现了巨大的数字，然后得到了巨大的斜率）

In [56]:
def sqrt(v):
    x = v / 2 # 估计初始值
    precision = 1 # 差值（精度）
    lr = 0.01 # 学习率
    i = 0
    while precision > 1e-8: # 精度足够小就认为合格
        i += 1
        fx = x ** 2 - v
        precision = abs(fx) # 把x2与v的值的差存为本次估算的精度，
        loss = fx ** 2
        dx = (x ** 2 - v) * 2 * 2 * x
        x = abs(x - dx*lr)
        if (i<5000 and not i % 100) or (not i % 500):
            print(f"precisionon:{precision}, dx:{dx}, x:{x}")
        if i > 20000:
            print("迭代超过20000次，退出")
    print(f'迭代{i}次')
    return x
sqrt(2) 

precisionon:7.026411985577852e-08, dx:-3.974738780109644e-07, x:1.4142135415057158
迭代112次


1.4142135597978454

### 牛顿法求解sqrt(v)
$x_{n+1} = x_n - \frac{f(x_n)}{f\prime(x_n)}$  
$f(x) = x^2 - v$

In [62]:
def sqrt(v):
    x = v / 2
    fx = x ** 2 - v
    i = 0
    while abs(fx) > 1e-8:
        i += 1
        dfx = 2 * x
        x = x - fx / dfx
        fx = x ** 2 - v
    print(f'迭代{i}次')
    return x
sqrt(177)

迭代7次


13.304134695650072

### 二分法求解sqrt(v)
$c = (a+b)/2$

In [70]:
def sqrt(v):
    a, b, i = 0, v, 0
    while True:
        i += 1
        x = (a + b) / 2
        fx = x ** 2 - v
        if abs(fx) < 1e-8:
            break
        elif fx > 0: # 数字过大，认定估算值在小的一半里
            b = x
        else:
            a = x # 否则假定在大的一半里
    print(f'迭代{i}次')
    return x
sqrt(17)

迭代33次


4.123105625039898

### 矩阵最小二乘法求解仿射变换矩阵M


一个矩形三个顶点`(0,0), (50, 0), (50, 50)`, 变换后为`(30, 30), (30, 130), (130, 130)`, 求其仿射矩阵。

我们分别设起始和结束矩阵的坐标为：$(a_x, a_y), (b_x, b_y), (c_x, c_y)$， 变换后的加一个prime（$ ^\prime$)符号，以此类推。  
要知道，一个3X2的矩阵是不可能右乘一个矩阵得到一个3X2的矩阵（只能左乘一个3X3的），  
然后，每一个新坐标，都是由原坐标的(x, y)经过变换得到(x', y‘），按教材的理论，即使是新坐标的X值，也是需要原坐标的(x, y)值参与过来进行变化的（乘以合适的系数），然后还要加上偏移的系数，以`x'`为例，应该是这样：

$$
a^\prime_x = a_x m_{00} + a_y m_{01} + m_{02} \\
$$
所以我们构造这个矩阵：

$$
\begin{bmatrix}
\color{red}{a_x} & \color{red}{a_y} & \color{red}1 \\
b_x & b_y & 1 \\
c_x & c_y & 1 \\
\end{bmatrix}
\begin{bmatrix}
\color{red}{m_{00}} \\ \color{red}{m_{01}} \\ \color{red}{m_{02}}
\end{bmatrix} = 
\begin{bmatrix}
\color{red}{a^\prime_x} \\ b^\prime_x \\ c^\prime_x
\end{bmatrix} \tag{红色部分即为上面的等式}
$$

这是把三个x给变换出来了，**其实你也可以认为这是把y给变换出来了**（因为原理一样，只是系数不同）。  
做到这一步，我们已经知道要如何求y坐标了，即我们只补一列的话，只能得到一个坐标的x值（或y值），要求另一半，根据坐标相乘的原理，看来只能把前三列置零，再把后三列复制进去了（__这样仿射矩阵也就变成6X1了__），其实就是上面矩阵乘法的重复，只不过交错一下形成x,y交错的排列：

$$
\begin{bmatrix}
a_x & a_y & 1 & 0 & 0 & 0 \\
0 & 0 & 0 & a_x & a_y & 1 \\
b_x & b_y & 1 & 0 & 0 & 0 \\
0 & 0 & 0 & b_x & b_y & 1 \\
c_x & c_y & 1 & 0 & 0 & 0 \\
0 & 0 & 0 & c_x & c_y & 1 
\end{bmatrix}
\begin{bmatrix}
m_{00} \\ m_{01} \\ m_{02} \\ m_{10} \\ m_{11} \\ m_{12}
\end{bmatrix} = 
\begin{bmatrix}
a^\prime_x \\ a^\prime_y \\ b^\prime_x \\ b^\prime_y \\ c^\prime_x \\ c^\prime_y \\
\end{bmatrix}
$$

原理当然就是把第一个公式补全：

$$
\begin{cases}
    \; a^\prime_x = a_x m_{00} + a_y m{01} + m_{02} \\
    \; a^\prime_y = a_x m_{10} + a_y m{11} + m_{12} \\
    \\
    \; b^\prime_x = b_x m_{00} + b_y m{01} + m_{02} \\
    \; b^\prime_y = b_x m_{10} + b_y m{11} + m_{12} \\
    \\
    \; c^\prime_x = c_x m_{00} + c_y m{01} + m_{02} \\
    \; c^\prime_y = c_x m_{10} + c_y m{11} + m_{12} \\
\end{cases}
$$

最小二乘的公式如下：

$$
\lVert A\beta - Y \rVert{^2_2} \quad A \in \mathbb{R}^{(m\times n+1)}, \beta \in \mathbb{R}^{(n+1)\times 1}, Y \in \mathbb{R}^{m\times 1}
$$

$$
\hat \beta = (A^TA)^{-1}A^TY
$$

[推导过程见此](https://iewaij.github.io/introDataScience/OLS.html)

我们把A和Y都做出来了，直接套用公式即可，为了编程方便，我们把前后矩阵设为A和B，仿射矩阵为M，就成了：

$$
M = (A^TA)^{-1}A^TB
$$

>奇异矩阵没有逆矩阵，$(A^TA)^{-1}$会出现无法求解的问题，也就是该方法对数据是有约束的。

In [115]:
A = [[0,0], [50, 0], [50, 50]]
B = [[30, 30], [130, 30], [130, 130]]

# 分别整理成上面说的6x6和6x1的矩阵
# 先定义变量保留6个坐标的值
(ax, ay), (bx, by), (cx, cy) = A
(ax1, ay1), (bx1, by1), (cx1, cy1) = B

A = np.array([
    [ax, ay, 1, 0, 0, 0],
    [0, 0, 0, ax, ay, 1],
    [bx, by, 1, 0, 0, 0],
    [0, 0, 0, bx, by, 1],
    [cx, cy, 1, 0, 0, 0],
    [0, 0, 0, cx, cy, 1]
])
B = np.array([ax1, ay1, bx1, by1, cx1, cy1]).reshape(6, 1) # 比手写6X1矩阵要省事
M = np.linalg.inv(A.T @ A) @ A.T @ B
M.reshape(2, 3)

array([[ 2.,  0., 30.],
       [ 0.,  2., 30.]])

### 梯度下降法求仿射矩阵M

根据上面的公式，每一个坐标均是由原坐标的变换再加上一个系数，所以其实是把原矩阵变成了3X3的矩阵（补1），  
所以，上面求出的仿射矩阵要验证的话这么验：

In [120]:
def print_lr(lr):
    print(f'set lr = {lr}')

theta = np.random.normal(size = (6,1))
lr = 3e-4
print_lr(lr)
schedule = {20000: 1e-5, 2800: 1e-6}

for i in range(30000):
    if i in schedule:
        lr = schedule[i]
        print_lr(lr)
        
    predict = A @ theta
    loss = 0.5 * np.sum((B - predict) ** 2)  # 此处应该把平方包起来再sum吧？ 待会试(结果没变)
    
    if not i % 3000:
        print(f'iter {i}, loss = {loss:g}')
    
    # 对loss求导， 外层是简单代数，内层是矩阵(d-Aθ/dθ = -AT)
    delta = A.T @ (predict - B)
    theta = theta - lr * delta
    
np.round(theta.reshape(2, 3))

set lr = 0.0003
iter 0, loss = 21064.9
set lr = 1e-06
iter 3000, loss = 163.891
iter 6000, loss = 162.911
iter 9000, loss = 161.937
iter 12000, loss = 160.968
iter 15000, loss = 160.006
iter 18000, loss = 159.049
set lr = 1e-05
iter 21000, loss = 155.279
iter 24000, loss = 146.24
iter 27000, loss = 137.726


array([[ 2., -0., 19.],
       [ 0.,  2., 19.]])